In [9]:
import numpy, matplotlib, pickle, pandas

#matplotlib.use ('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [4]:
results

NameError: name 'results' is not defined

In [10]:
def plot_sim_theory_diseased (outFile, diseases, results):

    ## Set up canvas
    h  = plt.figure (figsize=(7.5, 7.5))
    gs = gridspec.GridSpec (2, 1, height_ratios=[4, 1])
    #gs.update (bottom=0.1)

    ## Top plot: priority and hierarchical wait-time per condition
    axis = h.add_subplot (gs[0])

    yrange = [numpy.inf, -numpy.inf]
    for qtype in ['fifo', 'priority', 'hierarchical']:
        filtered_df = results[results.index.str.contains(qtype)]
        ## For fifo (without-CADt), only one data point
        if qtype == 'fifo':
            ylower = filtered_df.loc['fifo_non-interrupting', 'lowerSim']
            yupper = filtered_df.loc['fifo_non-interrupting', 'higherSim']
            yvalue = filtered_df.loc['fifo_non-interrupting', 'sim']
            yerror = [[yvalue - ylower], [yupper - yvalue]]
            axis.errorbar (1, filtered_df.loc['fifo_non-interrupting', 'sim'],
                           marker='x', ms=10, color='blue',
                           yerr=yerror, label='w/o CADt',
                           elinewidth=2, alpha=0.7, ecolor='blue', linestyle='')      
            axis.scatter (1, filtered_df.loc['fifo_non-interrupting', 'theory'],
                          marker='*', s=100, facecolors='none', edgecolors='blue', alpha=0.7) 
            if ylower < yrange[0]: yrange[0] = ylower
            if yupper > yrange[1]: yrange[1] = yupper
            continue
        ## For priority/hierarchical, it is per-condition
        xvalues, svalues, slowers, suppers, tvalues = [], [], [], [], []
        color = 'red' if qtype=='priority' else 'green'
        offset = -0.15 if qtype == 'priority' else +0.15
        for dindex, disease in enumerate (diseases):
            xvalues.append (dindex+2+offset)
            indexname = qtype + ' ' + disease
            svalues.append (filtered_df.loc[indexname, 'sim'])
            slowers.append (filtered_df.loc[indexname, 'lowerSim'])
            suppers.append (filtered_df.loc[indexname, 'higherSim'])
            tvalues.append (filtered_df.loc[indexname, 'theory'])
        svalues = numpy.array (svalues)
        slowers = numpy.array (slowers)
        suppers = numpy.array (suppers)
        yerror = [svalues - slowers, suppers - svalues]
        axis.errorbar (xvalues, svalues, marker='x', ms=10, color=color,
                       yerr=yerror, label=qtype + '; sim',
                       elinewidth=2, alpha=0.5, ecolor=color, linestyle='')      
        axis.scatter (xvalues, tvalues, marker='*', s=100, facecolors='none',
                      edgecolors=color, alpha=0.7, label=qtype + '; theory') 
        if min (slowers) < yrange[0]: yrange[0] = min (slowers)
        if max (suppers) > yrange[1]: yrange[1] = max (suppers)        

    ## Format plotting style
    #  x-axis
    axis.set_xlim (0, len (diseases)+2)
    xticks = numpy.arange (len (diseases)+2)
    axis.set_xticks (xticks)
    axis.set_xticklabels([])
    for xtick in axis.get_xticks():
        axis.axvline (x=xtick, color='gray', alpha=0.3, linestyle='--', linewidth=0.3)
    #  y-axis
    yrange[1] += 70
    axis.set_ylim (yrange)
    axis.set_ylabel ('Mean wait-time [min]')
    for ytick in axis.get_yticks():
        axis.axhline (y=ytick, color='gray', alpha=0.3, linestyle='--', linewidth=0.3)
    #  legend and title
    axis.legend (loc=1, ncol=2, prop={'size':12})
    axis.set_title ('Simulation & theory', fontsize=15)

    ## Bottom plot: time-savings
    axis = h.add_subplot (gs[1])

    for qtype in ['priority', 'hierarchical']:
        filtered_df = results[results.index.str.contains(qtype)]
        noAI_svalue = results.loc['fifo_non-interrupting', 'sim']
        noAI_tvalue = results.loc['fifo_non-interrupting', 'theory']
        ## For priority/hierarchical, it is per-condition
        xvalues, svalues, tvalues = [], [], []
        color = 'red' if qtype=='priority' else 'green'
        offset = -0.15 if qtype == 'priority' else +0.15
        for dindex, disease in enumerate (diseases):
            xvalues.append (dindex+2+offset)
            indexname = qtype + ' ' + disease
            svalues.append (filtered_df.loc[indexname, 'sim'] - noAI_svalue)
            tvalues.append (filtered_df.loc[indexname, 'theory'] - noAI_tvalue)
        axis.scatter (xvalues, svalues, marker='x', c=color, s=100, alpha=0.7, label='sim')    
        axis.scatter (xvalues, tvalues, marker='*', s=100, facecolors='none',
                      edgecolors=color, alpha=0.7) 

    ## Format plotting style
    #  x-axis
    axis.set_xlim (0, len (diseases)+2)
    xticks = numpy.arange (len (diseases)+2)
    axis.set_xticks (xticks)
    xticklabels = ['', 'no-AI'] + list (diseases)
    axis.set_xticklabels (xticklabels, fontsize=12)
    axis.tick_params (axis='x', labelsize=12)
    for xtick in axis.get_xticks():
        axis.axvline (x=xtick, color='gray', alpha=0.3, linestyle='--', linewidth=0.3)
    #  y-axis
    #axis.set_ylim (yrange)
    axis.set_ylabel ('time-savings\n[min]')
    for ytick in axis.get_yticks():
        axis.axhline (y=ytick, color='gray', alpha=0.3, linestyle='--', linewidth=0.3)
    #  legend and title
    #axis.legend (loc=1, ncol=1, prop={'size':12})

    h.savefig (outFile)
    plt.close('all')


In [11]:
with open ('AILVOICHExp4_hypoMixed.p', 'rb') as f:
    results = pickle.load (f)
f.close ()

results = pandas.DataFrame (results)
diseases = [rowname.split()[1] for rowname in results.index if 'hierarchical' in rowname]

plot_sim_theory_diseased ('plots/AILVOICHExp3_Mixed.pdf', diseases, results)

In [7]:
with open ('AILVOICHExp4_hypoOrdered.p', 'rb') as f:
    results = pickle.load (f)
f.close ()

results = pandas.DataFrame (results)
diseases = [rowname.split()[1] for rowname in results.index if 'hierarchical' in rowname]

plot_sim_theory_diseased ('plots/AILVOICHExp3_Ordered.pdf', diseases, results)

FileNotFoundError: [Errno 2] No such file or directory: 'AILVOICHExp4_hypoOrdered.p'